# Exercice 3 - Merry Christmhash
⚠️ Run this with an ipython kernel, not sage

In [1]:
import binascii
import struct
import operator
import random
import math

## Parameters

In [2]:
# avoids undesired parameters printing when commented
dummy_variable=1

### Tom

In [3]:

Q3a_M="If you are crying, fight against it! If you're regretting, walk forward! Only complaining on your misfortune, you're nothing but a common pig! (Ciel Phantomhive, Kuroshitsuji)"
Q3a_H=[1712935222, 2277794147, 2376157840, 3916784640, 1346939745, 2708679518, 85473800, 3788905359]
Q3a_K=[1943621227, 886196854, 2945353417, 208223340, 1212883955, 2473679821, 3054683082, 2029524489, 3963858188, 3339906788, 2823505815, 3350944332, 312194943, 3986347121, 2144670729, 741633462, 3663151569, 3544144315, 2690702324, 1848842070, 811599001, 510224092, 4281468752, 752009382, 1104123092, 3497858695, 2695667604, 1727413627, 3050579566, 510494887, 4285911224, 952491461, 4237080290, 10651148, 4015126669, 154644693, 1282241408, 2068274028, 2010214286, 3416638395, 2453720085, 3229980118, 2094058516, 1197555596, 922344893, 2154715712, 3908004108, 481673144, 1957904962, 3050061601, 541865778, 1006059256, 585392047, 1178853390, 2805354064, 972068900, 3754759061, 1435964078, 398864381, 2460187893, 1385904313, 3634606251, 2462881886, 3619520975]
Q3a_S=[[22, 16, 27], [21, 4, 23]]
Q3b_t="011e9faabb4e5381"
Q3b_d="384bba4b1dbd84001ccb55791cfcb7ac4da9f6f67174102751830195e3ca5fa6"
Q3b_H=[1410008662, 3165996059, 4265820587, 3228924364, 2876772144, 502571215, 1296440626, 3528054622]
Q3b_K2=[1660732939, 2978614430, 479083038, 4039110515, 503760146, 2160178725, 3371169074, 3023444970, 1832019271, 1110792947, 4275626369, 1675250051, 2784406696, 4257265674, 517985909, 3086983861, 3409296999, 1504386796, 374146206, 4055850561, 2556892257, 2255194630, 2066495719, 259846162, 593904479, 2868322264, 3146981800, 3913516318, 625630895, 1491933359, 1856807117, 3795630429, 3035508049, 3725788811, 2587675021, 4081590610, 1694559383, 2421525974, 2954023209, 3274021365, 556994319, 3840950079, 2748231069, 4293925912, 1934082025, 683379336, 3605902893, 3811110963, 3312404527, 562459431, 3079990482, 4161695928, 1789332738, 3133889965, 94188891, 1773848693]
Q3b_S=[[8, 1, 2], [9, 5, 13]]

dummy_variable=1

### Manon

## $H(m,H,K,S)$ implementation

In [4]:
# https://200ok.ch/posts/2018-12-09_unhexlify.html
def xxd(s):
    return binascii.hexlify(s).decode()

In [5]:
def pack(k,n):
    if k==32:
        return list(struct.pack('>L',n))
    elif k==64:
        return list(struct.pack('>Q',n))
    else:
        print("Error, k should be 32 or 64, here it's {}".format(k))
        return None

In [6]:
assert pack(32,1)==[0,0,0,1]
assert pack(64,1)==[0,0,0,0,0,0,0,1]

In [7]:
# we must make sure ^ and ~ operators are well defined as used later
for i in range(4096):
    r1 = random.randint(0,2**64)
    r2 = random.randint(0,2**64)
    assert r1^r2 == operator.xor(r1,r2)
    assert ~r1 == operator.invert(r1)

In [8]:
def compress(x,W,K,S):
    y = x
    for i in range(64):
        s0=0
        for j in range(0,3):
            # we make all our ops modulo 2^32 as y elements are expected to be
            s0 = (s0 ^ ((y[0]>>(S[0][j])) | (y[0]<<(32-S[0][j]))))%2**32
        s1=0
        for j in range(0,3):
            s1 = (s1 ^ ((y[4]>>(S[1][j])) | (y[4]<<(32-S[1][j]))))%2**32
        t1 = (y[4] & y[5] ^ (~y[4]) & y[6])%2**32
        t2 = (s1 + t1 + y[7] + K[i] + W[i])%2**32
        t3 = (y[0] & y[1] ^ y[0] & y[2] ^ y[1] & y[2])%2**32
        y = [(s0+t2+t3)%2**32,y[0],y[1],y[2],(y[3]+t2)%2**32,y[4],y[5],y[6]]
    return y

In [9]:
def extend(M):
    W = [0 for i in range(64)]
    for i in range(0,16):
        W[i] = M[i]
    for i in range(16,64):
        t1 = ((((W[i-15])>>7) | ((W[i-15])<<25)) ^ (((W[i-15])>>18) | ((W[i-15])<<14)) ^ (((W[i-15])>>3)))%2**32
        t2 = ((((W[i-2])>>17) | ((W[i-2])<<15)) ^ (((W[i-2])>>19) | ((W[i-2])<<13)) ^ (((W[i-2])>>10)))%2**32
        W[i] = (W[i-16]+W[i-7]+t1+t2)%2**32
    return W

In [10]:
def chunks(lis, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lis), n):
        yield lis[i:i + n]

In [11]:
def H(m,H,K,S):
    m = list(m)
    m_numberofbytes = len(m)
    l = (-m_numberofbytes-9)%64
    m = m + [0x80] + pack(64,0)*l + pack(64,8*m_numberofbytes)
    M = list(chunks(m,64))
    d = H
    L = len(M)
    for i in range(L):
        W = extend(M[i])
        y = compress(d,W,K,S)
        d = (d+y)
        d = [di % 2**32 for di in d]
        
    d_reduced = d[:8]
    d_out = []
    for di in d_reduced:
         d_out += pack(32,di)
    return d_out

In [12]:
digest = H(Q3a_M.encode(), Q3a_H, Q3a_K, Q3a_S)
digest

[102,
 25,
 81,
 54,
 135,
 196,
 97,
 99,
 141,
 161,
 74,
 144,
 233,
 117,
 100,
 0,
 80,
 72,
 171,
 97,
 161,
 115,
 43,
 94,
 5,
 24,
 58,
 8,
 225,
 214,
 27,
 143]

In [13]:
Q3a_d = binascii.hexlify(bytes(digest)).decode()
Q3a_d

'6619513687c461638da14a90e97564005048ab61a1732b5e05183a08e1d61b8f'

## Key recovery

In [15]:
def recover_k1(K2, t, H, d):
    return 0

In [27]:
eight_zeros = [0] * 8
K_test = eight_zeros + Q3a_K[8:]
print(K_test),

[0, 0, 0, 0, 0, 0, 0, 0, 3963858188, 3339906788, 2823505815, 3350944332, 312194943, 3986347121, 2144670729, 741633462, 3663151569, 3544144315, 2690702324, 1848842070, 811599001, 510224092, 4281468752, 752009382, 1104123092, 3497858695, 2695667604, 1727413627, 3050579566, 510494887, 4285911224, 952491461, 4237080290, 10651148, 4015126669, 154644693, 1282241408, 2068274028, 2010214286, 3416638395, 2453720085, 3229980118, 2094058516, 1197555596, 922344893, 2154715712, 3908004108, 481673144, 1957904962, 3050061601, 541865778, 1006059256, 585392047, 1178853390, 2805354064, 972068900, 3754759061, 1435964078, 398864381, 2460187893, 1385904313, 3634606251, 2462881886, 3619520975]


(None,)

In [26]:
print(Q3a_K)

[1943621227, 886196854, 2945353417, 208223340, 1212883955, 2473679821, 3054683082, 2029524489, 3963858188, 3339906788, 2823505815, 3350944332, 312194943, 3986347121, 2144670729, 741633462, 3663151569, 3544144315, 2690702324, 1848842070, 811599001, 510224092, 4281468752, 752009382, 1104123092, 3497858695, 2695667604, 1727413627, 3050579566, 510494887, 4285911224, 952491461, 4237080290, 10651148, 4015126669, 154644693, 1282241408, 2068274028, 2010214286, 3416638395, 2453720085, 3229980118, 2094058516, 1197555596, 922344893, 2154715712, 3908004108, 481673144, 1957904962, 3050061601, 541865778, 1006059256, 585392047, 1178853390, 2805354064, 972068900, 3754759061, 1435964078, 398864381, 2460187893, 1385904313, 3634606251, 2462881886, 3619520975]


In [28]:
digest_test = H(Q3a_M.encode(), Q3a_H, K_test, Q3a_S)
print(digest_test,)

[102, 25, 81, 54, 135, 196, 97, 99, 141, 161, 74, 144, 233, 117, 100, 0, 80, 72, 171, 97, 161, 115, 43, 94, 5, 24, 58, 8, 225, 214, 27, 143]


In [29]:
print(digest,)

[102, 25, 81, 54, 135, 196, 97, 99, 141, 161, 74, 144, 233, 117, 100, 0, 80, 72, 171, 97, 161, 115, 43, 94, 5, 24, 58, 8, 225, 214, 27, 143]


In [30]:
kzeros = [0]*64
digest_test2 = H(Q3a_M.encode(), Q3a_H, kzeros, Q3a_S)
print(digest_test2,)

[102, 25, 81, 54, 135, 196, 97, 99, 141, 161, 74, 144, 233, 117, 100, 0, 80, 72, 171, 97, 161, 115, 43, 94, 5, 24, 58, 8, 225, 214, 27, 143]
